In [1]:
import psycopg2
import random
import string
from datetime import datetime, timedelta

### DB connection

In [2]:
db_params = {
    "host": "localhost",
    "database": "python-vs-go",
    "user": "matthewbruner",
    "port": "5432",
}

try:
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    print("database connection successful...")

except (Exception, psycopg2.Error) as error:
    print("failed to connect...")

database connection successful...


### Create Tables

In [3]:
users = """
    CREATE TABLE IF NOT EXISTS users (
        id SERIAL PRIMARY KEY,
        username VARCHAR(20) UNIQUE,
        date_created DATE
    );
"""

followers = """
    CREATE TABLE IF NOT EXISTS followers (
        user_id INT,
        follower_id INT,
        PRIMARY KEY (user_id, follower_id),
        FOREIGN KEY (user_id) REFERENCES users(id),
        FOREIGN KEY (follower_id) REFERENCES users(id)
    );
"""

posts = """
    CREATE TABLE IF NOT EXISTS posts (
        id SERIAL PRIMARY KEY,
        user_id INT,
        post_date DATE,
        content VARCHAR(150),
        FOREIGN KEY (user_id) REFERENCES users(id)
    );
"""

likes = """
    CREATE TABLE IF NOT EXISTS likes (
        user_id INT,
        post_id INT,
        PRIMARY KEY (user_id, post_id),
        FOREIGN KEY (user_id) REFERENCES users(id),
        FOREIGN KEY (post_id) REFERENCES posts(id)
    );
"""

try:
    cursor.execute(users)
    cursor.execute(followers)
    cursor.execute(posts)
    cursor.execute(likes)
    connection.commit()
    print("tables successfully created...")

except (Exception, psycopg2.Error) as error:
    print("one or more tables failed to create...\n", error)


tables successfully created...


### Generate Dummy Users

In [13]:
users_records = []
num_records_to_add = 2

def generate_users_records(n):
    for _ in range(0,n):
        username = ''.join(random.choice(string.ascii_letters) for _ in range(random.randint(5, 20)))
        date_created = (datetime.now() - timedelta(days=3652) + timedelta(days=random.randint(0, (datetime.now() - (datetime.now() - timedelta(days=3652))).days))).strftime("%Y-%m-%d")
        users_records.append((username, date_created))

generate_users_records(num_records_to_add)

insert_records = """
    INSERT INTO users (username, date_created)
        VALUES (%s, %s)
        ON CONFLICT DO NOTHING;
"""

try:
    for record in users_records:
        cursor.execute(insert_records, record)
    connection.commit()
    print("Dummy records inserted successfully...")

except (Exception, psycopg2.Error) as error:
    print("Error inserting dummy records:", error)

Dummy records inserted successfully...


### Generate Followers

In [ ]:
follower_records = []
num_records_to_add = 1000000

def generate_follower_records(n):
    for _ in range(0,n):
        try:
            query = """
                SELECT id FROM users
            """
            cursor.execute(query)
            records = cursor.fetchall()

        except (Exception, psycopg2.Error) as error:
            print(error)

        user_id = random.randint(1, len(records) - 1)
        follower_id = user_id
        while follower_id == user_id:
            follower_id = random.randint(1, len(records) - 1)
        
        follower_records.append((user_id, follower_id))


generate_follower_records(num_records_to_add)

insert_records = """
    INSERT INTO followers (user_id, follower_id)
        VALUES (%s, %s)
        ON CONFLICT DO NOTHING;
"""

try:
    for record in follower_records:
        cursor.execute(insert_records, record)
    connection.commit()
    print("Dummy records inserted successfully...")

except (Exception, psycopg2.Error) as error:
    print("Error inserting dummy records:", error)

### Generate Posts

In [6]:
post_records = []
num_records_to_add = 10000

def generate_post_records(n):
    try:
        query = """
            SELECT id FROM users
        """
        cursor.execute(query)
        records = cursor.fetchall()

    except (Exception, psycopg2.Error) as error:
        print(error)
    for _ in range(0,n):

        user_id = random.randint(1, len(records) - 1)
        post_date = (datetime.now() - timedelta(days=3652) + timedelta(days=random.randint(0, (datetime.now() - (datetime.now() - timedelta(days=3652))).days))).strftime("%Y-%m-%d")
        content = ''.join(random.choice(string.ascii_letters) for _ in range(random.randint(5, 150)))
        post_records.append((user_id, post_date, content))


generate_post_records(num_records_to_add)

insert_records = """
    INSERT INTO posts (user_id, post_date, content)
        VALUES (%s, %s, %s);
"""

try:
    for record in post_records:
        cursor.execute(insert_records, record)
    connection.commit()
    print("Dummy records inserted successfully...")

except (Exception, psycopg2.Error) as error:
    print("Error inserting dummy records:", error)

Dummy records inserted successfully...


### Generate Likes

In [7]:
likes_records = []
num_records_to_add = 100000

def generate_likes_records(n):
    try:
        users_query = """
            SELECT count(id) FROM users
        """
        posts_query = """
            SELECT count(id) FROM posts
        """
        cursor.execute(users_query)
        user_count = cursor.fetchone()[0]
        cursor.execute(posts_query)
        post_count = cursor.fetchone()[0]

    except (Exception, psycopg2.Error) as error:
        print(error)

    for _ in range(0,n):
        user_id = random.randint(1, user_count)
        post_id = random.randint(1, post_count) 
        likes_records.append((user_id, post_id))


generate_likes_records(num_records_to_add)

insert_records = """
    INSERT INTO likes (user_id, post_id)
        VALUES (%s, %s)
        ON CONFLICT DO NOTHING;
"""

try:
    for record in likes_records:
        cursor.execute(insert_records, record)
    connection.commit()
    print("Dummy records inserted successfully...")

except (Exception, psycopg2.Error) as error:
    print("Error inserting dummy records:", error)

Dummy records inserted successfully...
